Columnwise estimation method for inverse Cov
---------------------------------------------

* The idea:

Ax_k = e_k ->  x_k = A^(-1)e_k

Solve x_k

* Thoughts

Pitäisi olla yksikäsitteinen ratk, koska n tuntematonta ja n yhtälöä

In [13]:
# imports

import numpy as np
import xarray as xr
import scipy.sparse.linalg 
invm = np.linalg.inv
gmres = scipy.sparse.linalg.gmres
npsolve = np.linalg.solve

ds_1 = xr.open_dataset("data/regions_verify_isotope_202112_cov.nc")
bio_1 = ds_1["covariance_bio"]
anth_1 = ds_1["covariance_anth"]

ds_2 = xr.open_dataset("data/regions_verify_202104_cov.nc")
bio_2 = ds_2["covariance_bio"]
anth_2 = ds_2["covariance_anth"]

M = anth_2.values

In [2]:
invM = invm(M)


np.linalg.solve
----------------
* ainakin näyttäisi tulevan samat sarakevektorit, mutta se ei (kai) käytä GMRESiä

In [19]:
i = 4 #which column to solve
e_i = np.zeros(len(M))
e_i[i]= 1.

s_i = np.linalg.solve(M, e_i)
print(s_i)
c_i = invM[:,i]

print(np.allclose(c_i, s_i))

[-4.11504027e-01 -4.28670605e-10  8.31936008e-09 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
True


scipy.sparse.linalg.gmres
-----------------------------

* needs a smaller tolerance than the default for the resulting vector to be close to the actual column of the inverse

In [20]:
g_i, exitCode = gmres(M,e_i, tol = 1e-12)

print(g_i)
print(exitCode)


[-4.11504027e-01 -4.28670908e-10  8.31936012e-09 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
0


Comparison of times
-------------------

* scipyn gmres a lot faster than numpy solve at least with tolerance 1e-12 or larger
* not as exact though (if we presume invm computes the correct inverse. (It could also be that invm uses the same np.linalg.solve and that is the reason it gives the same column as solve)

In [21]:
i = 5 #which column to solve
e_i = np.zeros(len(M))
e_i[i]= 1.

%timeit npsolve(M, e_i)
%timeit gmres(M,e_i, tol = 1e-12)

1.39 s ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
971 ms ± 17.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
